In [0]:
pip install mlflow

In [0]:
# import relevant functions
from pyspark.sql import functions as F
from pyspark.sql.functions import avg
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from sklearn.model_selection import train_test_split

In [0]:
# import data
circuits = spark.read.csv('s3://columbia-gr5069-main/raw/circuits.csv', header=True) 
constructor_results = spark.read.csv('s3://columbia-gr5069-main/raw/constructor_results.csv', header=True) 
constructor_standings = spark.read.csv('s3://columbia-gr5069-main/raw/constructor_standings.csv', header=True) 
constructors = spark.read.csv('s3://columbia-gr5069-main/raw/constructors.csv', header=True) 
driver_standings = spark.read.csv('s3://columbia-gr5069-main/raw/driver_standings.csv', header=True) 
drivers = spark.read.csv('s3://columbia-gr5069-main/raw/drivers.csv', header=True) 
lap_times = spark.read.csv('s3://columbia-gr5069-main/raw/lap_times.csv', header=True) 
pit_stops = spark.read.csv('s3://columbia-gr5069-main/raw/pit_stops.csv', header=True) 
qualifying = spark.read.csv('s3://columbia-gr5069-main/raw/qualifying.csv', header=True) 
races = spark.read.csv('s3://columbia-gr5069-main/raw/races.csv', header=True) 
results = spark.read.csv('s3://columbia-gr5069-main/raw/results.csv', header=True) 
seasons = spark.read.csv('s3://columbia-gr5069-main/raw/seasons.csv', header=True) 
sprint_results = spark.read.csv('s3://columbia-gr5069-main/raw/sprint_results.csv', header=True) 
status = spark.read.csv('s3://columbia-gr5069-main/raw/status.csv', header=True) 
winequality = spark.read.csv('s3://columbia-gr5069-main/raw/winequality-red_main.csv', header=True) 

In [0]:
display(results)

In [0]:
display(drivers)

In [0]:
display(races)

In [0]:
# combine results, races, and drivers to get necessary columns in one dataframe
drivers_results = results.join(races.select("raceId", "date"), on="raceId").join(drivers.select("driverId", "dob", "nationality"), on="driverId")

# take difference between date of race and date of driver's birth and divide by 365 to get years
drivers_results = drivers_results.withColumn("Age", F.datediff(drivers_results.date, drivers_results.dob)/365)

# turn age value into integer to get whole number for age of driver in years
drivers_results = drivers_results.withColumn("Age", drivers_results["Age"].cast(IntegerType()))

# select only relevant columns and display final dataframe
drivers_results = drivers_results.select("raceId", "driverId", "Age", "nationality", "position")
display(drivers_results)